In [1]:
!pip install catboost
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report

# Загрузка данных
data = pd.read_excel('Features.xlsx')
data['mood'] = data['mood'].replace('angry', 'sad')

# Предварительная обработка данных
le_mood = LabelEncoder()
data['mood'] = le_mood.fit_transform(data['mood'])

X = data.drop(['file', 'mood', 'key', 'scale'], axis=1)
y = data['mood']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определение гиперпараметров для поиска
param_grid = {
    'iterations': [500, 1000],
    'learning_rate': [0.001, 0.01],
    'depth': [4, 6],
    'l2_leaf_reg': [1, 3],
}

# Создание модели
model = CatBoostClassifier(verbose=0)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Выбираем лучшую модель и её параметры
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

print("Лучшие параметры:", best_params)

# Оценка точности
y_pred = best_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le_mood.classes_))

import joblib

joblib.dump(best_model, 'music_classifier.pkl')
joblib.dump(le_mood, 'le_mood.pkl')



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 9.3 MB/s eta 0:00:00
Лучшие параметры: {'depth': 6, 'iterations': 1000, 'l2_leaf_reg': 3, 'learning_rate': 0.01}
Accuracy: 0.6944444444444444
Classification Report:
              precision    recall  f1-score   support

       happy       0.66      0.59      0.62        46
     relaxed       0.66      0.57      0.61        40
         sad       0.72      0.80      0.76        94

    accuracy                           0.69       180
   macro avg       0.68      0.65      0.66       180
weighted avg       0.69      0.69      0.69       180



['le_mood.pkl']